<a href="https://colab.research.google.com/github/HugoKD/NLP/blob/main/Projet2_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Autocomplétion et génération de phrases avec des modèles de langue n-grammes

### Membres:

- Hugo Cadet
- Henri Ngo

## Description:

Ce projet est dédié à l'utilisation des n-grammes en traitement du langage naturel (NLP). Les n-grammes sont des séquences de n mots consécutifs dans un texte, et ils sont présent dans de nombreuses applications du NLP.


<a name='1'></a>
## 1.  Chargement et pré-traitement des données

<a name='1.1'></a>
### 1.1 Chargement des données

Les données qu'on a utilisé dans ce travail sont contenues dans le ichier [trump.txt](./trump.txt) <br>
Cf ce qui suit


In [ ]:
filename = "trump.txt"

with open(filename, 'r') as fichier:
    # Lire tout le contenu du fichier en une seule fois
    data = fichier.read()

data[:300]

'Thank you very much.\nWe had an amazing convention.\nThat was one of the best.\nI think it was one of the best ever.\nIn terms -- in terms of enthusiasm, in terms of I think what it represents, getting our word out.\nIvanka was incredible last night.\nShe did an incredible job.\nAnd so many of the speakers'

<a name='1.2'></a>
### 1.2  Segmentation

Pré-traitement des données en suivant les étapes suivantes:

1. Enlever les majuscules.
2. Remplacer les "\n" par des espaces
3. Séparer les données en phrases en utilisant les délimiteurs suivants `.`, `?` et `!` comme séparateur.
4. Enlever les signes de ponctuation (Attention de garder les espaces).
5. Enlever les phrases vides.
6. Segmenter les phrases avec la fonction nltk.word_tokenize()


In [ ]:
from os import replace
import nltk
import re
import string

nltk.download('punkt')


def preprocess(data):
    data = data.lower()
    data = data.replace('\n', ' ')
    sentences = re.split(r'!|\.|\?', data)
    sentences = [re.sub(r'[^\w\s]', '', sentence) for sentence in sentences]
    sentences = [sentence for sentence in sentences if sentence.strip()]
    tokens = [nltk.word_tokenize(sentence) for sentence in sentences]

    return tokens

data = preprocess(data)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# test
x = "Cats are independent.\nDogs are faithful."
preprocess(x)

[['cats', 'are', 'independent'], ['dogs', 'are', 'faithful']]

##### Sortie attendue

```CPP
[['cats', 'are', 'independent'], ['dogs', 'are', 'faithful']]
```

<a name='1.3'></a>
###  1.3 Création d'ensembles d'entraînement et de test

 **aléatoirement** 80% des données pour l'ensemble d'entrainement.
 <br> 20% pour l'ensemble de test.

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test = train_test_split(data, test_size=0.20, train_size=0.80, random_state=42)


In [ ]:
X_train[0]

['what', 'a', 'combination', 'those', 'two', 'are', 'right']

<a name='1.4'></a>
### 1.4 Construction du vocabulaire



In [ ]:
from collections import Counter


def build_voc(documents, threshold): #document : List ? , document : train_set ? ok for now
  jetons=[]
  cnt = Counter()
  for document in documents:
    for mot in document:
      cnt[mot] +=1
      if cnt[mot] >= threshold and mot not in jetons :
        jetons.append(mot)
  return jetons

voc=build_voc(X_train,2)

<a name='1.5'></a>
### 1.5 Mots hors vocabulaire


Gestion des mots hors vocabulaire (Out of Vocabulary) <b>OOV</b>.
Le pourcentage de mots inconnus dans l'ensemble de test est appelé le taux de mots <b> OOV </b>.

Pour gérer les mots inconnus lors de la prédiction, utilisez un jeton spécial 'unk' pour représenter tous les mots inconnus.


In [ ]:
import numpy as np

def replace_oov(tokenized_sentences, voc):
    token_sentences=[]
    for sentence in tokenized_sentences:
      Sentence = []
      for mot in sentence:
        if mot in voc :
          Sentence.append(mot)
        else:
          mot = '<unk>'
          Sentence.append(mot)
      token_sentences.append(Sentence)
    return token_sentences


In [ ]:
tokenized_sentences = [["cats", "sleep"], ["mice", "eat"], ["cats", "and", "mice"]]
vocabulary = build_voc([["cats", "sleep"], ["mice", "eat"], ["cats", "and", "mice"]], 2)
tmp_replaced_tokenized_sentences = replace_oov(tokenized_sentences, vocabulary)
print(f"Phrase initiale:")
print(tokenized_sentences)
print(f"Phrase segmentée avec'<unk>':")
print(tmp_replaced_tokenized_sentences)

Phrase initiale:
[['cats', 'sleep'], ['mice', 'eat'], ['cats', 'and', 'mice']]
Phrase segmentée avec'<unk>':
[['cats', '<unk>'], ['mice', '<unk>'], ['cats', '<unk>', 'mice']]


##### Sortie attendue
```CPP
Phrase initiale:
[['cats', 'sleep'], ['mice', 'eat'], ['cats', 'and', 'mice']]
Phrase segmentée avec '<unk>':
[['cats', '<unk>'], ['mice', '<unk>'], ['cats', '<unk>', 'mice']]
```

<a name='2'></a>
## 2. Modèles de langue n-gramme



Dans cette section, on developpe un modèle de langue n-grammes. Nous allons utiliser la formule:

$$ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_t)}{C(w_{t-1}\dots w_{t-n})} \tag{2} $$

- La fonction $C(\cdots)$ représente le nombre d'occurrences de la séquence donnée.
- $\hat{P}$ signifie l'estimation de $P$.


<a name='2.1'></a>
### 2.1 Fréquence des n-grammes



On commence par mettre en œuvre une fonction qui calcule la fréquence des n-grammes pour un nombre arbitraire $n$.

En pré-traitant la phrase en ajoutant $n$ marqueurs de début de phrase "\<s\>" pour indiquer le commencement de la phrase.

- Par exemple, dans un modèle bigramme (N=2), la séquence devrait commencer avec deux jetons de début "\<s\>\<s\>". Ainsi, si la phrase est "J'aime la nourriture", modifiez-la pour devenir "\<s\>\<s\> J'aime la nourriture".
- Ajoutez aussi un jeton de fin "\<e\>" pour que le modèle puisse prédire quand terminer une phrase.
    
    
Ensuite, chaque n-gram est stocké en dictionnaire où :
- La clé de chaque paire clé-valeur dans le dictionnaire est un tuple de n mots (et non une liste).
- La valeur dans la paire clé-valeur est le nombre d'occurrences.

In [ ]:
from collections import Counter
def count_n_grams(data, n, start_token='<s>', end_token = '<e>'):
    cnt = Counter()
    for sentence in data:
      L=[]
      sentence = [start_token]*n + sentence + [end_token]
      i=0
      while i+n<= len(sentence):
        ngram = tuple(sentence[i:i+n])
        cnt[ngram]+=1
        i+=1
    return cnt

In [ ]:
sentences = [['i', 'have', 'a', 'mouse'],
             ['this', 'mouse', 'likes', 'cats']]
print("unigramme:")
print(count_n_grams(sentences, 1))
print("Bigrammes:")
print(count_n_grams(sentences, 2))

unigramme:
Counter({('<s>',): 2, ('mouse',): 2, ('<e>',): 2, ('i',): 1, ('have',): 1, ('a',): 1, ('this',): 1, ('likes',): 1, ('cats',): 1})
Bigrammes:
Counter({('<s>', '<s>'): 2, ('<s>', 'i'): 1, ('i', 'have'): 1, ('have', 'a'): 1, ('a', 'mouse'): 1, ('mouse', '<e>'): 1, ('<s>', 'this'): 1, ('this', 'mouse'): 1, ('mouse', 'likes'): 1, ('likes', 'cats'): 1, ('cats', '<e>'): 1})


Sortie attendue:

```CPP
Unigrammes:
{('<s>',): 2, ('i',): 1, ('have',): 1, ('a',): 1, ('mouse',): 2, ('<e>',): 2, ('this',): 1, ('likes',): 1, ('cats',): 1}
Bigrammes:
{('<s>', '<s>'): 2, ('<s>', 'i'): 1, ('i', 'have'): 1, ('have', 'a'): 1, ('a', 'mouse'): 1, ('mouse', '<e>'): 1, ('<s>', 'this'): 1, ('this', 'mouse'): 1, ('mouse', 'likes'): 1, ('likes', 'cats'): 1, ('cats', '<e>'): 1}

```

<a name='2.2'></a>
### 2.2 Estimé du maximum de vraisemblance MLE

#### 2.2.1 Calcul de probabilité pour un mot


Ensuite, estimation de la probabilité d'un mot étant donnés les 'n' mots précédents avec les fréquences obtenues.

$$ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_t)}{C(w_{t-1}\dots w_{t-n})} \tag{2}$$


La fonction prend en entrée:

- word : le mot dont on veut estimer la probabilité
- previous_n_gram : le n-gramme précédent, sous forme de tuple
- n_gram_counts: Un dictionnaire où la clé est le n-gramme et la valeur est la fréquence de ce n-gramme.
- n_plus1_gram_counts: dictio n-gramme précédent plus le mot actuel.


In [ ]:

def estimate_probability(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts):
    frqce_n = n_gram_counts[previous_n_gram]
    frqce_plus1 = n_plus1_gram_counts[previous_n_gram+[word]]

    if frqce_plus1 > 0:
        return frqce_plus1/frqce_n
    else:
        return 0

Nous ne verrons jamais assez de données pour estimer ces
probabilités avec précision. Supposons que le n-gramme ("le", "petit") n'apparaît jamais dans les données d'entraînement. Si vous appelez estimate_probability("chat", ("le", "petit"), n_gram_counts, n_plus1_gram_counts), la fonction renverra une probabilité de 0, ce qui n'est pas réaliste. De plus, la fonction aurait une exeption d'erreur si jamais elle trouvera une fréquence nulle en dénominateur.

<a name='2.3'></a>
### 2.3  Lissage add-k


$$ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_n) + k}{C(w_{t-1}\dots w_{t-n}) + k|V|} \tag{3} $$


In [ ]:
def estimate_probability_smoothing(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
    frqce_n = n_gram_counts[tuple(previous_n_gram)]
    frqce_plus1 = n_plus1_gram_counts[tuple(previous_n_gram+[word])]
    return (frqce_plus1+ k)/(frqce_n +k*vocabulary_size)



In [ ]:
# test
sentences = [['i', 'have', 'a', 'mouse'],
             ['this', 'mouse', 'likes', 'cats']]
unique_words = list(set(sentences[0] + sentences[1]))

bigram_counts = count_n_grams(sentences, 2)
trigram_counts = count_n_grams(sentences, 3)
tmp_prob = estimate_probability_smoothing("have", ['<s>', 'i'], bigram_counts, trigram_counts, len(unique_words), k=1)

print(f" La probabilité de 'have' étant donné le mot précédent 'i' est: {tmp_prob:.4f}")

 La probabilité de 'have' étant donné le mot précédent 'i' est: 0.2500


<a name='2.4'></a>
### 2.4 Calcul des probabilités des n-grammes

#### 2.4.1. Estimation des probabilités
Création de la fonction estimate_probabilities qui calcule pour chaque mot du vocabulaire la probabilité d'être généré en utilisant la fonction avec lissage add-k.


Cette fonction prends en entrée:
- previous_n_gram
- n_gram_counts
- n_plus1_gram_counts
- vocabulary: le vocabulaire
- k: la constante de lissage

La fonction retourne un dictionnaire ayant pour clés tous les mots du vocabulaire ainsi que leur probabilité d'être générés

In [ ]:
def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):
    dico = {}
    vocabulary.append('<e>')
    for word in vocabulary: #add <e>
      frqce = estimate_probability_smoothing(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, len(vocabulary), k)
      dico[word]= frqce
    return dico

In [ ]:
# test
sentences = [['i', 'have', 'a', 'mouse'],
             ['this', 'mouse', 'likes', 'cats']]

unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
estimate_probabilities(["a"], unigram_counts, bigram_counts, unique_words, k=1)

{'cats': 0.1111111111111111,
 'mouse': 0.2222222222222222,
 'this': 0.1111111111111111,
 'a': 0.1111111111111111,
 'have': 0.1111111111111111,
 'i': 0.1111111111111111,
 'likes': 0.1111111111111111,
 '<e>': 0.1111111111111111}

##### Sortie attendue

```CPP
{'likes': 0.1111111111111111,
 'have': 0.1111111111111111,
 'this': 0.1111111111111111,
 'i': 0.1111111111111111,
 'mouse': 0.2222222222222222,
 'a': 0.1111111111111111,
 'cats': 0.1111111111111111,
 '<e>': 0.1111111111111111}
```

#### 2.4.2. Probabilités étant donné un contexte

On affiche maintenant les probabilités des tri-grammes étant donné le context "i will" en utilisant les données d'entraînement, .limit(10)

In [ ]:
bigram_counts = count_n_grams(X_train, 2)
trigram_counts = count_n_grams(X_train, 3)

x = sorted(estimate_probabilities(["i","will"], bigram_counts, trigram_counts, voc, k=1).items(),key=lambda x:x[1],reverse=True)

In [ ]:
print(x[:10])

[('tell', 0.0070981916511745815), ('fix', 0.005239141456819334), ('fight', 0.005239141456819334), ('be', 0.0050701368936961295), ('never', 0.004225114078080108), ('say', 0.003718100388710495), ('not', 0.0025350684468480648), ('ask', 0.0023660638837248605), ('also', 0.0018590501943552475), ('work', 0.001521041068108839)]


##### Sortie attendue

```CPP
[('tell', 0.0070981916511745815),
 ('fix', 0.005239141456819334),
 ('fight', 0.005239141456819334),
 ('be', 0.0050701368936961295),
 ('never', 0.004225114078080108),
 ('say', 0.003718100388710495),
 ('not', 0.0025350684468480648),
 ('ask', 0.0023660638837248605),
 ('also', 0.0018590501943552475),
 ('work', 0.001521041068108839)]
```

<a name='3'></a>
## 3. Perplexité

Dans cette section, on génère le score de perplexité pour évaluer votre modèle sur l'ensemble de test. **A Approfondir**

Pour calculer le score de perplexité d'une phrase sur un modèle n-gramme :

$$PP(W) =\sqrt[N]{ \prod_{t=1}^{N} \frac{1}{P(w_t | w_{t-n} \cdots w_{t-1})} } \tag{4.1}$$

où N = le nombre de jeton dans la phrases incluant le jeton \<e\>
et P = la probabilité de générer le jeton $w_t$

Plus les probabilités sont élevées, plus la perplexité sera basse.

<a name='3.1'></a>
### 3.1. Calcul de la perplexité
On créer la fonction `calculate_perplexity`, qui pour une phrase donnée, nous donne le score de perplexité.

In [ ]:
import math

def calculate_perplexity(sentence, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):

    # Set n to the order of the n-gram model (for example, 1 for unigram, 2 for bigram, etc.)
    counter_items = list(n_gram_counts.items())
    n = len(counter_items[0][0])

    # Add tokens "<s>" "<e>" to sentence
    words = ['<s>']*n + sentence + ['<e>']

    # initialize perplexity
    perplexity = 1.0

    # Calculate the number of words in the sentence
    num_words = len(words)

    for i in range(n - 1, num_words-1):
        # Extract the previous n-gram (context) and the target word
        previous_n_gram = words[i - (n - 1) : i+1]
        target_word = words[i+1]

        # Calculate the conditional probability using n-gram and (n+1)-gram counts
        prob = estimate_probability_smoothing(target_word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0)

        # Update perplexity
        perplexity *= 1.0 / prob

    perplexity_finale = math.pow(perplexity, 1/(num_words-n))

    return perplexity_finale

In [ ]:
# test

sentences = [['i', 'have', 'a', 'mouse'],
             ['this', 'mouse', 'likes', 'cats']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)


perplexity = calculate_perplexity(sentences[0],
                                         unigram_counts, bigram_counts,
                                         len(unique_words), k=1.0)
print(f"Perplexité de la première phrase : {perplexity:.4f}")


Perplexité de la première phrase : 4.1930


<a name='3.2'></a>
### 3.2. Perplexité sur une phrase d'entraînement


In [ ]:
# test de modèle bi-grammes :

voc_1 = build_voc(X_train,1000)

bigram_counts = count_n_grams(X_train, 2)
trigram_counts = count_n_grams(X_train, 3)


perplexity = calculate_perplexity(X_train[0], bigram_counts, trigram_counts, len(voc_1), k=0.01)
print(f"Perplexité de la première phrase : {perplexity:.4f}")

Perplexité de la première phrase : 31.4048


In [ ]:
# test de modèle tri-grammes :

trigram_counts = count_n_grams(X_train, 3)
quadrigram_counts = count_n_grams(X_train, 4)


perplexity = calculate_perplexity(X_train[0],
                                         trigram_counts, quadrigram_counts,
                                         len(voc_1), k=0.01)
print(f"Perplexité de la première phrase : {perplexity:.4f}")

Perplexité de la première phrase : 30.2854


In [ ]:
# test de modèle quadri-grammes :

quadrigram_counts = count_n_grams(X_train, 4)
quintigram_counts = count_n_grams(X_train, 5)


perplexity = calculate_perplexity(X_train[0],
                                         trigram_counts, quadrigram_counts,
                                         len(voc_1), k=0.01)
print(f"Perplexité de la première phrase : {perplexity:.4f}")

Perplexité de la première phrase : 30.2854


<a name='3.3'></a>
### 3.3. Perplexité du corpus de test

#### 3.3.1. On peut maintenant calculer et afficher la perplexité des modèles bi-grammes, tri-grammes et quadri-grammes sur votre corpus de test. K=1 ici.

Pour calculer la perplexité d'un corpus de *m* phrases, il suffit de suivre la formule suivante :

Soit $N$ le nombre total de jetons dans le corpus de test C et $N_i$ le nombre de jetons dans la phrase i.

$$Perplexity(C) = \Big(\frac{1}{P(s_1, ..., s_m)}\Big)^{1/N}$$
$$P(s_1, ..., s_m) = \prod_{i=1}^{m} p(s_i)$$
$$p(s_i) = \prod_{t=1}^{N_i} \hat{P}(w_t | w_{t-n} \cdots w_{t-1})$$

Puisqu'il s'agit d'un multiplication de probabilités (situées entre 0 et 1), le produit devient nul très rapidement. C'est pourquoi il est plus efficace d'effectuer une transformation vers un espace logarithmique pour transformer les multiplications en addition. Cela donne ainsi la formule suivante:

$$LogPerplexity(C) = 2^{-\frac{1}{N} \sum_{k=1}^{m} log_{2} \; p(s_k)}$$


In [ ]:
def calculate_perplexity_corpus(corpus, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
  perplexity_sentences = 0
  T = 0

  for phrase in corpus :
    T += len(phrase)+1
    perplexity_sentences += math.log2(calculate_perplexity(phrase, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k))

  Log_Perplexity_corpus = math.pow(2, (-1)*perplexity_sentences/T)

  return (Log_Perplexity_corpus)


In [ ]:
# n_gram_counts = {('<s>', 'quick'): 1, ('the', 'quick'): 1, ('quick', 'brown'): 1, ('brown', 'fox'): 1, ('jumps', 'over'): 1, ('over', 'the'): 1, ('the', 'lazy'): 1, ('lazy', 'dog'): 1, ('dog', '<e>'): 1}
# n_plus1_gram_counts = { ('<s>', '<s>', 'the', ): 1, ('<s>', 'the', 'quick'): 1, ('the', 'quick', 'brown'): 1, ('quick', 'brown', 'fox'): 1, ('jumps', 'over', 'the'): 1, ('over', 'the', 'lazy'): 1, ('the', 'lazy', 'dog'): 1, ('lazy', 'dog', '<e>'): 1}

train_corpus = [["the", "quick", "brown", "fox"], ["jumps", "over", "the", "lazy", "dog"]]
n_gram_counts = {('<s>', '<s>'): 2, ('<s>', 'the'): 1, ('<s>', 'jumps'): 1, ('the', 'quick'): 1, ('quick', 'brown'): 1, ('brown', 'fox'): 1, ('fox', '<e>'): 1, ('jumps', 'over'): 1, ('over', 'the'): 1, ('the', 'lazy'): 1, ('lazy', 'dog'): 1, ('dog', '<e>'): 1}
n_plus1_gram_counts = {('<s>', '<s>', '<s>', ): 2, ('<s>', '<s>', 'the', ): 1, ('<s>', 'the', 'quick'): 1,  ('<s>', '<s>', 'jumps', ): 1, ('<s>', 'jumps', 'over'): 1, ('the', 'quick', 'brown'): 1, ('quick', 'brown', 'fox'): 1, ('brown', 'fox', '<e>'): 1, ('jumps', 'over', 'the'): 1, ('over', 'the', 'lazy'): 1, ('the', 'lazy', 'dog'): 1, ('lazy', 'dog', '<e>'): 1}

vocabulary = ["the", "quick", "brown", "fox", "jumps", "over", "lazy", "dog", "<e>"]

N = 5
V = len(vocabulary)

test_corpus = [["the", "fox"], ["jumps"]]


# Complétez le calcul de la perplexité avec k=1

# On a pris les 2 derniers gram_counts
n_gram_counts = Counter(n_gram_counts)
n_plus1_gram_counts = Counter(n_plus1_gram_counts)

calculate_perplexity_corpus(test_corpus, bigram_counts, trigram_counts, V, 1.0)


0.5170269930330363

#### Sortie attendue

    Perplexité du corpus de test:  7.708920690856638

In [ ]:
# Calculez mainenant la perplexité de votre corpus de test

voc_2 = build_voc(X_test,2)

bigram_counts = count_n_grams(X_test, 2)
trigram_counts = count_n_grams(X_test, 3)

calculate_perplexity_corpus(X_test, bigram_counts, trigram_counts, len(voc_2), 1.0)

0.6411413282064495

> On attends d'avoir des perplexités plus élevées que celles obtenues sur l'ensemble d'entraînement, vu que généralement les modèles performent bien en phase d'entraînement qu'en phase de test. D'autre part, on pourrait expliquer ces résultats obtenus du fait que le corpus de test est plus simple, la taille du corpus de test est aussi tellement petit que celle d'entraînement ce qui facilite d'obtenir des performances en test beaucoup mieux qu'en entraînement.

<a name='4'></a>
## 4. Construction d'un modèle d'auto-complétion

Dans cette dernière partie, on utilise enfin les modèles n-grammes construits aux numéros précédents afin de faire un modèle d'autocomplétion.

<a name='4.1'></a>
### 4.1 Suggestion d'un mot à partir d'un préfixe


La première étape sera de construire une fonction qui suggère un mot à partir des premiers caractères entrés par un utilisateur, considérant un seul type de n-gramme.  

La fonction `suggest_word` retourne le mot le plus probable avec la probabilité associée

In [ ]:
def suggest_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0, prefixe=""):
      dico =estimate_probabilities(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k) #pb proba uniforme
      keys = list(dico.keys())
      idx = None
      for key in keys:
        if key.startswith(prefixe):
          if idx == None :
            idx = key
          elif dico[idx] < dico[key]:
            idx = key
      if idx == None : return "error",0
      else : return idx, dico[idx]

In [ ]:
# test
sentences = [['i', 'have', 'a', 'mouse'],
             ['this', 'mouse', 'likes', 'cats']]

unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
trigram_counts = count_n_grams(sentences, 3)

previous_tokens = ["i", "have"]
# tmp_suggest1= suggest_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0)

tmp_suggest1_1 = suggest_word(previous_tokens, bigram_counts, trigram_counts, unique_words, k=1.0)

print(f" avec les mots précédents 'i have',\n\t le mot suggéré est `{tmp_suggest1_1[0]}` avec la probabilité {tmp_suggest1_1[1]:.4f}")

print()


tmp_starts_with = 'm'
tmp_suggest2 = suggest_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0, prefixe=tmp_starts_with)
print(f"avec les mots précédents 'i have', et une suggestion qui commence par `{tmp_starts_with}`\n\t le mot suggéré est : `{tmp_suggest2[0]}` avec une probabilité de {tmp_suggest2[1]:.4f}")

 avec les mots précédents 'i have',
	 le mot suggéré est `a` avec la probabilité 0.2000

avec les mots précédents 'i have', et une suggestion qui commence par `m`
	 le mot suggéré est : `mouse` avec une probabilité de 0.1000


### Sortie attendue

```CPP
avec les mots précédents 'i have',
	 le mot suggéré est `a` avec la probabilité 0.2222

avec les mots précédents 'i have', et une suggestion qui commence par `m`
	 le mot suggéré est : `mouse` avec une probabilité de 0.1111


```

<a name='4.2'></a>
### 4.2 Suggestions multiples

Afin de suggérer plusieurs mots à l'utilisateur, une stratégie que l'on peut utiliser est de retourner un ensemble de mots suggérés par plusieurs types de modèles n-grammes.

En utilisant la fonction `suggest_word` du numéro précédent, on complete la fonction `get_suggestions` qui retourne les suggestions des modèles n-grammes passés en paramètre.

La fonction get_suggestions prends en paramètres:
- previous_n_gram: le n-gramme précédent, sous forme de tuple
- n_gram_counts_list: une liste de n-grammes dans l'ordre suivant [unigrammes, bigrammes, trigrammes, quadrigrammes, ...]
- vocabulary_size: la taille du vocabulaire
- k: la constante de lissage (entre 0 et 1)
- prefixe: Le début du mot que l'on veut prédire, "" si au aucun préfixe

In [ ]:
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, prefixe=""):

  L=[]
  tmp_suggest_multip = []
  for i in range(len(n_gram_counts_list)-1):
    tmp_suggest_multip.append(suggest_word(previous_tokens, n_gram_counts_list[i], n_gram_counts_list[i+1], vocabulary, k=1.0, prefixe=""))

  # trier en ordre décroissant
  tmp_suggest_multip_sorted = sorted(tmp_suggest_multip, key=lambda item: item[1], reverse=True)

  return list(set([tmp_suggest_multip_sorted[0][0], tmp_suggest_multip_sorted[1][0]]))

In [ ]:
# test
sentences = [['i', 'have', 'a', 'mouse'],
             ['this', 'mouse', 'likes', 'cats']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
trigram_counts = count_n_grams(sentences, 3)
quadgram_counts = count_n_grams(sentences, 4)
qintgram_counts = count_n_grams(sentences, 5)

n_gram_counts_list = [unigram_counts, bigram_counts, trigram_counts, quadgram_counts, qintgram_counts]
previous_tokens = ["i", "have"]
tmp_suggest3 = get_suggestions(previous_tokens, n_gram_counts_list, unique_words, k=1.0)

print(f"Etant donné les mots i have, je suggère :")
display(tmp_suggest3)

{'cats': 0.125, 'i': 0.125, 'have': 0.125, 'mouse': 0.125, 'likes': 0.125, 'this': 0.125, 'a': 0.125, '<e>': 0.125}
{'cats': 0.1, 'i': 0.1, 'have': 0.1, 'mouse': 0.1, 'likes': 0.1, 'this': 0.1, 'a': 0.2, '<e>': 0.1}
{'cats': 0.1, 'i': 0.1, 'have': 0.1, 'mouse': 0.1, 'likes': 0.1, 'this': 0.1, 'a': 0.1, '<e>': 0.1}
{'cats': 0.09090909090909091, 'i': 0.09090909090909091, 'have': 0.09090909090909091, 'mouse': 0.09090909090909091, 'likes': 0.09090909090909091, 'this': 0.09090909090909091, 'a': 0.09090909090909091, '<e>': 0.09090909090909091}
Etant donné les mots i have, je suggère :


['cats', 'a']

Sortie obtenue : Etant donné les mots i have, je suggère :
['cats', 'a']

##### Sortie attendue

```CPP
Etant donné les mots i have, je suggère :
['a']
```

<a name='4.3'></a>
### 4.3 Autocomplétion

Il est maintenant temps de combiner nos fonctions afin de créer le modèle d'autocomplétion. En utilisant le jeu de données d'entraînement, on calcule la fréquence des n-grammes allant de 1 à 5 et utilisez la fonction *get_suggestions* afin de suggérer des mots.

In [ ]:
import ipywidgets as widgets
from IPython.display import display


text_input = widgets.Text(placeholder="Entrez votre text ici...")

suggestions_label = widgets.Label(v Autocomplétion et génération de phrases avec des modèles de langue n-grammesalue="Suggestions: ")

def update_suggestions(change):
    texte_actuel = change["new"]

    # TODO

    top_suggestions = ["word1", "word2"]
    suggestions_label.value = "Suggestions: " + ", ".join(top_suggestions)

text_input.observe(update_suggestions, names="value")

display(text_input)
display(suggestions_label)

<a name='5'></a>
## 5. Modèle de génération de phrases

Dans cette partie on construit un modèle de génération de phrases en utilisant les n-grammes.


La stratégie de modèle N-gramme, faite en 2, a un contexte fixe et limité dépendant de la taille des N-grammes. Elle ne tiennent pas compte du contexte global ou d'informations non locales, ce qui la rend moins efficaces pour comprendre la signification complète d'un mot ou d'une phrase. Du coup, elle sera moins capables de généraliser efficacement sur des mots ou des séquences de mots qui ne se sont pas produits exactement dans les données d'entraînement.

<a name='5.1'></a>

### 5.1 Génération stochastique de mots

On recode la fonction suggest_word afin d'utiliser une suggestion stochastique. Autrement dit, au lieu de retourner le mot le plus probable, la fonction génére le mot suivant selon sa probabilité.

Par exemple si le mot 'like' a la probabilité 0.25 d'être généré, alors il sera retourné 25% du temps.


In [ ]:

def suggest_word_with_probs(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):

    # Estimez les probabilités des mots candidats
    probabilities = estimate_probabilities(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k)

    # le 2ème paramètre k = 1  est pour choisir enfin un seul mot :
    return random.choices(list(probabilities.keys()), list(probabilities.values()), k = 1)


<a name='5.2'></a>
### 5.2 Générations de phrases

#### 5.2.1. Génération stochastique
Finalement, on complète maintenant la fonction `generate_sentence` qui génère une phrase longue de n_words en appelant votre nouvelle fonction `suggest_words_with_probs`. La génération doit s'arrêter si le modèle génère un jeton de fin de phrase.


In [ ]:
def generate_sentence(n_words, n_gram_counts, n_plus1_gram_counts, vocabulary, k=0.0001):

  # Set n to the order of the n-gram model (for example, 1 for unigram, 2 for bigram, etc.)
  n = len(next(iter(n_gram_counts)))
  S = ['<s>']*n
  P = S
  while(len(S)<n_words):
    w = suggest_word_with_probs(P, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0)
    S = S + w
    if w == ['<e>']:
      return S
    else:
      P = S[len(S)-n:len(S)]
  return S

#### 5.2.2. Test sur des n-grammes
On teste ensuite votre fonction avec des trigrammes et des 5-grammes.

In [ ]:
sentences_exp = [['i', 'have', 'a', 'mouse', 'in', 'my','house'],
             ['this', 'mouse', 'likes', 'cats', 'and', 'dogs']]
unique_words_exp = list(set(sentences_exp[0] + sentences_exp[1]))

unigram_counts = count_n_grams(sentences_exp, 1)
bigram_counts = count_n_grams(sentences_exp, 2)
trigram_counts = count_n_grams(sentences_exp, 3)
quadrigram_counts = count_n_grams(sentences_exp, 4)
quintigram_counts = count_n_grams(sentences_exp, 5)

generate_sentence(6, bigram_counts, trigram_counts, unique_words_exp, k=0.0001)

['<s>', '<s>', 'likes', 'have', 'my', 'have']

In [ ]:
generate_sentence(10, quadrigram_counts, quintigram_counts, unique_words_exp, k=0.0001)

['<s>', '<s>', '<s>', '<s>', 'a', 'mouse', 'likes', 'my', 'my', 'house']

>On remarque que la génération du mot le plus fréquent a diminué, dans notre cas "mouse". Théoriquement si le lissage est excessif, la probabilité de chaque N-gramme tend à être plus uniforme. Cela signifie que les N-grammes rares deviennent plus probables et les N-grammes fréquents deviennent moins probables. Dans le but d'améliorer la situation, on peut envisager à priori un réglage du paramètre k : On essaie différentes valeurs de k jusqu'à avoir de meilleurs résultats.

#### 5.2.4.  Question sur la valeur de la constante kn si elle est trop petite :

>Les problèmes envisageable en ce cas sont :

*   Possibilité d'avoir des probabilités nulles
*   Les N-grammes peu fréquents, observés seulement une ou quelques fois, afficheront des probabilités très faibles, ce qui peut conduire à des erreurs significatives lorsque ces séquences apparaissent dans le texte.
*   Une perplexité élevée, ce qui se traduit par la difficulté du modèle à fournir des estimations précises de probabilité pour les phrases, car il assignera des probabilités très faibles à de nombreuses séquences.



<a name='5.3'></a>
### 5.3. Amélioration de la génération stochastique de mots

#### 5.3.1. Amélioration stochastique

Comme on a pu l'observer, la génération stochastique, bien qu'elle soit efficace pour générer des phrases différentes, a tendance à ne pas générer des phrases toujours cohérentes. On propose aisi une amélioration de la méthode `suggest_word` avec la méthode `suggest_word_new` permettant de générer des phrases plus cohérentes.

>

*   On choisira la bonne valeur de k suivant la meilleure performance. Premièrement, On a constaté expérimentalement dans la question 3 que dans notre contexte, Threshold = 1000 genère de bons résultats quand même.
*   On va ajuster la taille du vocabulaire suivant la meilleure performance.




##### b) Implémentation de la méthode proposée

In [ ]:
voc_2 = build_voc(X_train,1000)

bigram_counts = count_n_grams(X_train, 2)
trigram_counts = count_n_grams(X_train, 3)
quatrigram_counts = count_n_grams(X_train, 4)
quintigram_counts = count_n_grams(X_train, 5)

In [ ]:
k_values = [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0]
# Perform cross-validation to find the best value of k
best_k = None
best_perplexity = float('inf')

for k in k_values:
    perplexity = calculate_perplexity_corpus(X_train, bigram_counts, trigram_counts, len(voc_2), k)
    print(f'k = {k}: Perplexity = {perplexity}')

    if perplexity < best_perplexity:
        best_perplexity = perplexity
        best_k = k
print(f'Best k: {best_k}')

k = 0.0001: Perplexity = 0.7687053173040973
k = 0.001: Perplexity = 0.7687053173040973
k = 0.01: Perplexity = 0.7687053173040973
k = 0.1: Perplexity = 0.7687053173040973
k = 0.5: Perplexity = 0.7687053173040973
k = 1.0: Perplexity = 0.7687053173040973
Best k: 0.0001


Vu que dans notre cas, tous les k performent identiquement ! On prend donc une valeur recommandée dans la littérature k = 0.1

In [ ]:
threshold_values = [100, 500, 1000, 1500, 2000]
# Perform cross-validation to find the best value of threshold
best_threshold = None
best_perplexity = float('inf')

for threshold in threshold_values:
  voc_2 = build_voc(X_train,threshold)
  bigram_counts = count_n_grams(X_train, 2)
  trigram_counts = count_n_grams(X_train, 3)

  perplexity = calculate_perplexity_corpus(X_train, bigram_counts, trigram_counts, len(voc_2), k=0.1)
  print(f'threshold = {threshold}: Perplexity = {perplexity}')

  if perplexity < best_perplexity:
        best_perplexity = perplexity
        best_threshold = threshold
print(f'Best threshold: {best_threshold}')

threshold = 100: Perplexity = 0.6911507979595707
threshold = 500: Perplexity = 0.7444506894024
threshold = 1000: Perplexity = 0.7687053173040973
threshold = 1500: Perplexity = 0.785504586883338
threshold = 2000: Perplexity = 0.7925368532942627
Best threshold: 100


On trouve que la taille adéquate du vocabulaire est celle avec un seuil de répition minimal (Threshold) = 100

In [ ]:
# reproduire le vocabulaire avec le nouveau seuil Threshold = 100 :
voc_2 = build_voc(X_train,100)

bigram_counts = count_n_grams(X_train, 2)
trigram_counts = count_n_grams(X_train, 3)
quatrigram_counts = count_n_grams(X_train, 4)
quintigram_counts = count_n_grams(X_train, 5)

In [ ]:
# paramètre k est changé à 0.1,
# taille de vocabulaire a changé aussi pour un seuil de 100 :

def suggest_word_new(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=0.1):
  # Estimez les probabilités des mots candidats
    probabilities = estimate_probabilities(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k)

    return random.choices(list(probabilities.keys()), list(probabilities.values()), k = 1)

#### 5.3.2. Génération améliorée
Finalement :

In [ ]:
def generate_sentence_new(n_words, n_gram_counts, n_plus1_gram_counts, vocabulary, k=0.1):
  # Set n to the order of the n-gram model (for example, 1 for unigram, 2 for bigram, etc.)
  n = len(next(iter(n_gram_counts)))
  S = ['<s>']*n
  P = S
  while(len(S)<n_words):
    w = suggest_word_new(P, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0)
    S = S + w
    if w == ['<e>']:
      return S
    else:
      P = S[len(S)-n:len(S)]
  return S

#### 5.3.3. Test sur des n-grammes

In [ ]:
generate_sentence_new(8, bigram_counts, trigram_counts, voc_2, k=0.1)

['<s>', '<s>', 'thank', 'you', '<e>']

In [ ]:
generate_sentence_new(12, quadrigram_counts, quintigram_counts, voc_2, k=0.1)

['<s>',
 '<s>',
 '<s>',
 '<s>',
 'we',
 'need',
 'law',
 'theres',
 'like',
 'really',
 'deal',
 'number']